In [117]:
import numpy
import random
from collections import defaultdict
import urllib
import math
import random
import collections
import string

In [2]:
def parse(path):
  file = open(path, 'r')
  for l in file:
    yield eval(l)

print("Reading data...")
products = list(parse("C:\Users\Moi\Downloads\reviews.json"))
# http://jmcauley.ucsd.edu/data/amazon/qa/qa_Tools_and_Home_Improvement.json.gz
qa = list(parse("C:\Users\Moi\Downloads\qa.json"))
print("done")

Reading data...
done


In [100]:
reviews = defaultdict(lambda: [])
for product in products:
    reviews[product["asin"]].append(product["reviewText"])

In [101]:
def tf(term, document):
    count = collections.defaultdict(int)
    for word in document.split():
        count[word] += 1

    return count[term]/len(document.split())

In [112]:
# docDict is the union of a question and the set of all reviews for said question.
def idf(term, docSet):
    count = 0
    for doc in docSet:
        if term in doc.lower():
            count += 1
    return math.log(len(docSet)/(count+1))

In [132]:
def feature(document, docSet):
    document = document.lower()
    document = ''.join([c for c in document if not (c in string.punctuation)])
    
    feat = collections.defaultdict(int)
    for term in document.split():
        feat[term] = tf(term, document) * idf(term, docSet)
    return feat

In [133]:
# queryFeat is a feature vector for the query and reviewFeat is the feature vector for the review
def cosineSimilarity(queryFeat, reviewFeat):
    # Find the words the 2 dictionaries have in common
    querySet = set(queryFeat.keys())
    reviewSet = set(reviewFeat.keys())
    allWords = querySet.union(reviewSet)
    
    # Find the cosine similarity
    numerator = 0
    mag1 = 0
    mag2 = 0
    for word in allWords:
        numerator = numerator + queryFeat[word] * reviewFeat[word]
        mag1 = mag1 + queryFeat[word]**2
        mag2 = mag2 + reviewFeat[word]**2
    if mag1 > 0 and mag2 > 0:
        return (numerator/((mag1*mag2)**0.5))
    else:
        return -1

In [137]:
def randomQuestion(qa, reviews):
    numReviews = 0
    while numReviews < 100:
        question = random.choice(qa)
        questionText = question['question']
        asin = question['asin']

        numReviews = len(reviews[asin])
    
    # reviewsSet is list of all review text for this product
    reviewsSet = reviews[asin]
    # docDict is union of questionText and reviewsDoc
    docSet = list(reviewsSet)
    docSet.append(questionText)
    
    # Make dictionaries for the query and review
    queryFeat = feature(questionText, docSet)
    cosSimList = {review : cosineSimilarity(queryFeat, feature(review, docSet)) for review in reviewsSet}
    cosSimList = sorted(cosSimList.items(), key=lambda x:x[1])
    
    reviewSample = [review[0] for review in cosSimList[:5]]
    reviewSample = reviewSample + [review[0] for review in cosSimList[-5:]]
    
    print(questionText)
    print(cosSimList[-3:])
    
    return (asin, questionText, reviewSample)
    
    

In [138]:
with open("C:\Users\Moi\Downloads\out.csv", 'w') as the_file:
    for i in range(0,2):
        asin, question, reviewSample = randomQuestion(qa, reviews)
        for review in reviewSample:
            the_file.write(asin)
            the_file.write(',')
            the_file.write(question.replace(',', ''))
            the_file.write(',')
            the_file.write(review.replace(',', ''))
            the_file.write('\n')

What size battery will this take. AA or AAA? I am not familiar with what an "18650" is.
[("The Ultrafire 502b flashlight is just ok.  It requires the the 18650 rechargeable battery.  Alternately the Ultrafire Cree XML-T6 is a brighter inexpensive 5mode 18650 flashlight.  This unit cannot focus and is not as bright as the XML-T6.  Overall it's ok but there are better options.If you go with any flashlight using and 18650 battery consider the Obtronix protected 18650 3400mah rechargable batteries with the Nitecore charger.", 0.11558124376240016), ("I have about 50 led flashlights and always looking for one brighter that the other. I bought this for it's size to fit in my pocket but I was amazed at the brightness and the quality of this little flashlight. This little thing out performed some of my dual battery led lights so I am ordering a couple more for my mom and dad.I highly recommend getting a digital charger that you can see the charging in digital numerals. Those have a overcharge s

Can this one stand on its own? I want to put in on my night stand and NOT stick to a wall, is it possible?
[('I travel a lot and have always had to leave a hotel bathroom light on so I could find my way in the dark of a strange room.  I took one of these on a month-long trip to Europe last summer and it worked perfectly.  You just place the small unit on a counter or table which is between you and the bathroom.  When you stand up it the light goes on and you can walk safely.  I have recommended them for all my friends.', 0.11013975562285927), ('These work fine but do NOT stick on well.', 0.14353199051737145), ("Just stand them on the floor....light up fast and stay lit long enough to get where you're going. They work.", 0.16737877117620345)]
Has anyone else had trouble with the flashlight not always turning on? Not sure what's wrong with it because I just got it.
[("Pros-Waterproof as advertised-Powerful beam with good spread, not sure on battery life yetCons-Feels cheap.  I haven't dr

wow, lots of the same questions over and over. new one - is the angle of the light adjustable? it appears to only point straight out?
[('I was unsure of this products lighting. But when I installed this and tested at night wow this is bright. It lights up the whole area. The batteries are removable and you can charge these with the proper charge for the battery type. Save yourself 4 days of charging using the solar panel as recommended by manufacture.', 0.07559907058896266), ("Pros:Nice, bright light - better than expected.Adjustable settings for: 1. what constitutes dusk when the light begins to motion sense; 2. how sensitive the IR detector is to movement, and; 3. how long the light remains on.Selectable modes for auto, on, or off.IR sensor is adjustable up / down and left / right, separate from light.Power cord from solar cell is very long and gives you more flexibility on installation locations.Cons:Mounting hardware (not the units themselves) is not galvanized or otherwise weather